# Wildfire Drone Surveying
## Description
Imagine you are working for wildfire prevention and management. You have access to a fleet of eight drones to monitor the forest. You want to optimize your operations according to some constraints. You are limited by both the drone technology and the requirements of the surveying. 

You are given the following information:
1. The list of drones
2. The periods of surveying, divided in weeks
  * Example: MayWeek1, MayWeek2…
3. The number of drones needed for each week
4. The maintenance schedule for each drone – maintenance makes the drone unavailable for that week

## Requirements
### Technological
The drones require a periodical maintenance. The maintenance team has a fixed schedule, defined a priori and given to you as an input. While on maintenance, the drone cannot be used. Because the drones use a lithium battery, you would like to balance how much each drone is used. For example, instead of drone A being used continuously and drone B being used only a few times, you would prefer to use drone A and drone B the same amount of times as to reduce the cost of replacing batteries.

### Natural
For surveying purposes, each month of the year has a different requirement of drones – the riskier the month, the more drones you need to survey the forest properly. 


For questions on this assignment, please contact: andre.oliveira@ucalgary.ca

In [ ]:
%pip install gurobipy
%pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 32.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The `weeks` list lists the weeks by name. Note that because of how python works, the weeks are indexed from 0 to 27 instead of from 1 to 28. For more information about indexing lists in python, read [here](https://www.hackerearth.com/practice/notes/samarthbhargav/a-quick-intro-to-indexing-in-python/).

In [ ]:
weeks = [
    "April1", "April2", "April3", "April4", 
    "May1", "May2",  "May3",  "May4", 
    "June1", "June2", "June3", "June4", 
    "July1", "July2", "July3", "July4", 
    "August1", "August2", "August3", "August4", 
    "September1", "September2", "September3", "September4",
    "October1", "October2", "October3", "October4"]


The `drones` list is a list of available drones. Note that because of how python works, the drones index inside the list go from 0 to 7 instead of from 1 to 8.

In [ ]:
drones = ["Drone1", "Drone2", "Drone3", "Drone4", "Drone5", "Drone6", "Drone7", "Drone8"]

The `availabilityMatrix` links the drone with the week. Each column is a week, and each row is a drone.

> Indented block



In [ ]:
availabilityMatrix = [
#Week 0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
     [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0], # Drone1
     [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0], # Drone2
     [0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1], # Drone3
     [1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0], # Drone4
     [0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1], # Drone5
     [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1], # Drone6
     [1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0], # Drone7
     [1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1]  # Drone8
]

This `availability` dictionary allows for easy consultation. For example:
```availability["Drone1", "May2"]``` would return True if the drone is available for that day or False if the drone is unavailable

In [ ]:
availability = {(d, w): availabilityMatrix[j][i]
                for i, w in enumerate(weeks)
                for j, d in enumerate(drones)}

The `requirementsList` shows how many drones are needed to survey each week. Note that the weeks are indexed from 0 to 27 instead of from 1 to 28.

In [ ]:
#              Week 0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
requirementsList = [6, 4, 7, 5, 6, 6, 5, 5, 6, 6, 8, 7, 7, 8, 7, 7, 6, 6, 7, 8, 8, 7, 6, 8, 6, 5, 6, 6]

The `requirements` dictionay allows for easy consultation. For example: `requirements["May2"]` would return 3 because that is how many drones are needed for that week.

In [ ]:

requirements = {weeks[i]: w for i, w in enumerate(requirementsList)}

#Exercise 1:
Respecting the maintenance schedule and number of drones needed for each week, make a model that optimizes the following objectives:
1. Minimize the total number of drones missing for survey. For example, if in May2 you needed 5 drones and you had only 3, this week you were missing 2 drones. If the next week you needed 6 drones and you had 3, you were missing another 3, for a total of 5.
2. Minimize the difference between the maximum and minimum number of weeks of operation among all drones.

For this exercise, show the following:
1. For each week, print a string in the following format:
  * {Week name}: {Number of drones used}/{Number of drones needed}
  * If it helps debugging your code, you can also print which drones are allocated for each week.
2. Print the most used drone and the least used drone, along the number of times they were used.


HINT: To set multiple objectives, use: model.setObjectiveN(...., index=0, weight=, name='...')


In [ ]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("Drone Scheduling Exercise 1")

# Implement your model here

#adding variables
drones_used = model.addVars(drones, vtype=GRB.BINARY, name="Drones Used")


Restricted license - for non-production use only - expires 2024-10-28


#Exercise 2:
Imagine now you are allowed to also choose when each drone will have maintenance (where the 0 go in the availability matrix). However, each drone must have a minimum number of maintenances. 
There are a total of 28 weeks. You can choose which week each drone will be in maintenance. Start with 2 weeks of maintenance per drone (two 0 in each row in the availability matrix) and iterate all the way to 14 in steps of size 1.

For this exercise, show the following:
1. Plot a graph where the x-axis is the number of maintenances per drone over the period, going from 2 to 14, and the y-axis has two values, the total number of missing and the difference between how many times the most and least used drones were used.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

model = gp.Model("Drone Scheduling Exercise 2")

# Implement your model here

#Exercise 3:
Instead of forcing the drones to go through maintenance a certain number of times during the entire period, in this exercise you should try to optimize it.
In this exercise, you will also define when each drone goes through maintenance. However, instead of fixing a number of times you need to maximize the number of times as an optimization objective.

For this exercise, show the following:
1. Change your model to optimize the number of maintenances.
2. Iterate through the values for weights given in the exercise. The weights are as follows: `i` is the weight of the objective to minimize the total number of drone-weeks missed, `j` is the weight of the objective to minimize the difference between the number of max and min worked drones, and `k` is the weight of the objective to maximize the number of maintenance weeks. For your convenience, the code snippet has the weights in a data structure for easy iteration.

Answer the following questions:
  1. Is there any combination of weights that results in 0 total drone-weeks missed and 0 imbalance of drone usage? If yes, which weights?
  2. What is the highest number of maintenances that still allows for 0 total drone-weeks missed?
  3. What is the highest number of maintenances that still allows for 0 imbalance of drone usage?

In [ ]:
import gurobipy as gp
from gurobipy import GRB

weights = [
    (0.0, 0.0, 1.0),
    (0.05, 0.05, 0.9),
    (0.1, 0.1, 0.8),
    (0.15, 0.15, 0.7),
    (0.2, 0.2, 0.6),
    (0.25, 0.25, 0.5),
    (0.3, 0.3, 0.4),
    (0.35, 0.35, 0.3),
    (0.4, 0.4, 0.2),
    (0.45, 0.45, 0.1),
    (0.5, 0.5, 0.0)]
for i, j, k in weights:
  model = gp.Model("Drone Scheduling Exercise 3")
  # Implement your model here